In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
app =  pd.read_csv('../data/application_train.csv')
credit_card_balance =  pd.read_csv('../treated_data/credit_card_balance.csv')
POS_CASH_balance =  pd.read_csv('../treated_data/POS_CASH_balance.csv')
installments_payments =  pd.read_csv('../treated_data/installments_payments.csv')


In [4]:
credit_card_balance = credit_card_balance[credit_card_balance['MONTHS_BALANCE']>=-80]
POS_CASH_balance = POS_CASH_balance[POS_CASH_balance['MONTHS_BALANCE']>=-80]
installments_payments = installments_payments[installments_payments['MONTHS_BALANCE']>=-80]

In [12]:
credit_card_balance.isna().sum()

SK_ID_PREV                      0
SK_ID_CURR                      0
MONTHS_BALANCE                  0
AMT_DRAWINGS_POS_CURRENT      664
AMT_PAYMENT_CURRENT         30365
dtype: int64

In [13]:
df = pd.merge(app[['SK_ID_CURR', 'TARGET']], POS_CASH_balance, how='left')
# df = pd.merge(df, credit_card_balance, how='left')
df = pd.merge(df, installments_payments, how='left')
df = df.fillna(df.mean())

In [15]:
df = df.fillna(df.mean())

In [16]:
df.isna().sum()

SK_ID_CURR               0
TARGET                   0
SK_ID_PREV               0
MONTHS_BALANCE           0
CNT_INSTALMENT           0
CNT_INSTALMENT_FUTURE    0
DAYS_DELAYED             0
DAYS_DELAYED.1           0
dtype: int64

In [40]:
y = app['TARGET']

In [21]:
df = df.drop(['TARGET'], axis=1)

In [22]:
df = df.pivot_table(values=list(set(df.columns) - set(['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE'])), index='SK_ID_CURR', columns='MONTHS_BALANCE')


In [23]:
df

CNT_INSTALMENT                                                  \
MONTHS_BALANCE          -80.0 -79.0 -78.0 -77.0 -76.0 -75.0 -74.0 -73.0 -72.0   
SK_ID_CURR                                                                      
100002                    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
100003                    NaN   NaN   NaN  12.0  12.0  12.0  12.0  12.0  12.0   
100004                    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
100006                    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
100007                    NaN   NaN   NaN  10.0  10.0  10.0  10.0  10.0  10.0   
...                       ...   ...   ...   ...   ...   ...   ...   ...   ...   
456251                    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
456252                    6.0   6.0   6.0   6.0   6.0   NaN   NaN   NaN   NaN   
456253                    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
456254                    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
456255                    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   

                      ... DAYS_DELAYED.1                                   \
MONTHS_BALANCE -71.0  ...          -10.0      -9.0       -8.0       -7.0    
SK_ID_CURR            ...                                                   
100002           NaN  ...      17.000000  19.000000  15.000000  19.000000   
100003          12.0  ...            NaN        NaN        NaN        NaN   
100004           NaN  ...            NaN        NaN        NaN        NaN   
100006           NaN  ...      11.826665   4.000000   5.000000   5.000000   
100007          10.0  ...       3.000000   3.000000   4.000000   2.000000   
...              ...  ...            ...        ...        ...        ...   
456251           NaN  ...            NaN  11.826665  11.826665  27.000000   
456252           NaN  ...            NaN        NaN        NaN        NaN   
456253           NaN  ...            NaN        NaN        NaN        NaN   
456254           NaN  ...      26.000000  13.913333  20.000000  23.000000   
456255           NaN  ...       1.333333   0.500000   2.200000  17.333333   

                                                              
MONTHS_BALANCE      -6.0  -5.0       -4.0  -3.0  -2.0  -1.0   
SK_ID_CURR                                                    
100002          19.000000  15.0  18.000000  14.0  12.0  24.0  
100003                NaN   NaN        NaN   NaN   NaN   NaN  
100004                NaN   NaN        NaN   NaN   NaN   NaN  
100006           8.413333  18.0   9.413333   3.0   1.0   1.0  
100007           4.000000   2.0   2.000000   2.0   1.0   0.0  
...                   ...   ...        ...   ...   ...   ...  
456251          42.000000  44.0  46.000000  46.0  41.0   8.0  
456252                NaN   NaN        NaN   NaN   NaN   NaN  
456253                NaN   NaN        NaN   NaN   NaN   NaN  
456254          21.000000  18.0  16.000000  18.0  19.0  19.0  
456255          15.500000   4.0  15.500000   2.0  10.0   NaN  

[307511 rows x 324 columns]

In [27]:
df = df.values.reshape((307511,81,4))

In [30]:
df = np.nan_to_num(df)

In [41]:
y = y.to_numpy()

In [35]:
# np.savetxt('../data/X_train.csv', df, delimiter=',')
# np.savetxt('../data/Y_train.csv', y, delimiter=',')

ValueError: Expected 1D or 2D array, got 3D array instead

In [39]:
len(y)

8576306

In [44]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [42]:
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.33, random_state=42)

In [45]:
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(4, return_sequences = True),
    tf.keras.layers.LSTM(2, return_sequences = False)
])

In [46]:
model.compile(optimizer='adam', 
              loss='mean_squared_error', 
              metrics='accuracy')

In [ ]:
history = model.fit(x=X_train, y=y_train, epochs= 50, batch_size=500, validation_data=(X_test,y_test) )

Epoch 1/50
413/413 [==============================] - 4s 10ms/step - loss: 0.0739 - accuracy: 0.5490 - val_loss: 0.0732 - val_accuracy: 0.7238
Epoch 2/50
413/413 [==============================] - 4s 10ms/step - loss: 0.0739 - accuracy: 0.5461 - val_loss: 0.0732 - val_accuracy: 0.4482
Epoch 3/50
413/413 [==============================] - 6s 14ms/step - loss: 0.0739 - accuracy: 0.5621 - val_loss: 0.0732 - val_accuracy: 0.5645
Epoch 4/50
413/413 [==============================] - 5s 11ms/step - loss: 0.0739 - accuracy: 0.5656 - val_loss: 0.0732 - val_accuracy: 0.6592
Epoch 5/50
413/413 [==============================] - 4s 9ms/step - loss: 0.0739 - accuracy: 0.5597 - val_loss: 0.0732 - val_accuracy: 0.6652
Epoch 6/50
413/413 [==============================] - 5s 13ms/step - loss: 0.0739 - accuracy: 0.5809 - val_loss: 0.0732 - val_accuracy: 0.5690
Epoch 7/50
413/413 [==============================] - 5s 13ms/step - loss: 0.0739 - accuracy: 0.5433 - val_loss: 0.0732 - val_accuracy: 0.5310


In [ ]:
loss_train = history.history['train_loss']
loss_val = history.history['val_loss']
epochs = range(1,35)
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
cre_fill = credit_card_balance.fillna('mean')

In [ ]:
cre_fill.astype(int)

In [6]:

print(len(set(app['SK_ID_CURR'])))
print(len(set(credit_card_balance['SK_ID_CURR'])))
print(len(set(POS_CASH_balance['SK_ID_CURR'])))
print(len(set(installments_payments['SK_ID_CURR'])))

307511
86905
285213
288410


In [6]:
# values =list(set(installments_payments.columns) - set(['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE']))

In [7]:
POS_CASH_balance.columns

Index(['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE', 'CNT_INSTALMENT',
       'CNT_INSTALMENT_FUTURE', 'NAME_CONTRACT_STATUS', 'SK_DPD',
       'SK_DPD_DEF'],
      dtype='object')

In [9]:
credit_card_balance = credit_card_balance.pivot_table(values=list(set(credit_card_balance.columns) - set(['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE'])), index='SK_ID_CURR', columns='MONTHS_BALANCE')

POS_CASH_balance = POS_CASH_balance.pivot_table(values=list(set(POS_CASH_balance.columns) - set(['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE'])), index='SK_ID_CURR', columns='MONTHS_BALANCE')

installments_payments = installments_payments.pivot_table(values=list(set(installments_payments.columns) - set(['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE'])), index='SK_ID_CURR', columns='MONTHS_BALANCE')

In [24]:
POS_CASH_balance

CNT_INSTALMENT                                                \
MONTHS_BALANCE            -80  -79  -78   -77   -76   -75   -74   -73   -72   
SK_ID_CURR                                                                    
100002                    NaN  NaN  NaN   NaN   NaN   NaN   NaN   NaN   NaN   
100003                    NaN  NaN  NaN  12.0  12.0  12.0  12.0  12.0  12.0   
100004                    NaN  NaN  NaN   NaN   NaN   NaN   NaN   NaN   NaN   
100006                    NaN  NaN  NaN   NaN   NaN   NaN   NaN   NaN   NaN   
100007                    NaN  NaN  NaN  10.0  10.0  10.0  10.0  10.0  10.0   
...                       ...  ...  ...   ...   ...   ...   ...   ...   ...   
456251                    NaN  NaN  NaN   NaN   NaN   NaN   NaN   NaN   NaN   
456252                    6.0  6.0  6.0   6.0   6.0   NaN   NaN   NaN   NaN   
456253                    NaN  NaN  NaN   NaN   NaN   NaN   NaN   NaN   NaN   
456254                    NaN  NaN  NaN   NaN   NaN   NaN   NaN   NaN   NaN   
456255                    NaN  NaN  NaN   NaN   NaN   NaN   NaN   NaN   NaN   

                      ... SK_DPD_DEF                                          \
MONTHS_BALANCE   -71  ...        -10  -9   -8   -7   -6   -5   -4   -3   -2    
SK_ID_CURR            ...                                                      
100002           NaN  ...        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
100003          12.0  ...        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
100004           NaN  ...        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
100006           NaN  ...        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
100007          10.0  ...        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
...              ...  ...        ...  ...  ...  ...  ...  ...  ...  ...  ...   
456251           NaN  ...        NaN  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
456252           NaN  ...        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
456253           NaN  ...        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
456254           NaN  ...        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
456255           NaN  ...        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

                     
MONTHS_BALANCE  -1   
SK_ID_CURR           
100002          0.0  
100003          NaN  
100004          NaN  
100006          0.0  
100007          0.0  
...             ...  
456251          0.0  
456252          NaN  
456253          NaN  
456254          0.0  
456255          NaN  

[285213 rows x 320 columns]

In [22]:
credit_card_balance.shape

(86905, 1520)

In [25]:
POS_CASH_balance.shape

(285213, 320)

In [11]:
priit

NameError: name 'priit' is not defined

In [18]:
installments_payments.shape[1]/81

6.0

In [19]:
# credit_card_balance= credit_card_balance.values.reshape((86905,80,19))
# POS_CASH_balance= POS_CASH_balance.values.reshape((285213,80,4))
installments_payments= installments_payments.values.reshape((288410,81,6))


In [21]:
installments_payments.keys

AttributeError: 'numpy.ndarray' object has no attribute 'keys'

In [ ]:
df = installments_payments[installments_payments['SK_ID_CURR'].isin(app['SK_ID_CURR'])]

In [ ]:
df.shape

In [ ]:
df = pd.merge(df, POS_CASH_balance, how='left')
# df.head(5)

In [ ]:
df=pd.merge(df, credit_card_balance, how = 'left')
# df.head(5)

In [ ]:
df = df[df['MONTHS_BALANCE']>-80]

In [ ]:
df.shape

In [ ]:

del credit_card_balance, POS_CASH_balance ,installments_payments
gc.collect()

In [ ]:
# df = df[df['SK_ID_CURR'].isin(app['SK_ID_CURR'])]

In [ ]:
values = list(set(df.columns) - set(['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE']))

In [ ]:
df_pivot = df.pivot_table(values=values, index='SK_ID_CURR', columns='MONTHS_BALANCE')

In [ ]:
# import pandas as pd
# import numpy as np
# import gc
# credit_card_balance =  pd.read_csv('../data/credit_card_balance.csv')
# POS_CASH_balance =  pd.read_csv('../data/POS_CASH_balance.csv')
# installments_payments =  pd.read_csv('../data/installments_payments.csv')
# installments_payments['MONTHS_BALANCE'] = (installments_payments['DAYS_INSTALMENT']/30).round().astype(int)
# df = pd.merge(POS_CASH_balance, installments_payments,  left_on=(['SK_ID_PREV','SK_ID_CURR','MONTHS_BALANCE']), right_on=(['SK_ID_PREV','SK_ID_CURR','MONTHS_BALANCE']), how='outer')
# df=pd.merge(df, credit_card_balance, how = 'outer')
# del credit_card_balance, POS_CASH_balance ,installments_payments
# gc.collect()
# values = list(set(df.columns) - set(['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE']))
# df_pivot = df.pivot_table(values=values, index='SK_ID_CURR', columns='MONTHS_BALANCE').to_json('../treated_data/timeseries.json')